# Introduction to Neural Force Field with SIGOPT

This Jupyter Notebook is an introduction in using the `nff` package together with the hyperparameter optimization tool SIGOPT. This tutorial uses the same code and dataset as `01_training`, thus if you have not done that one, it is highly recommended to check it out.

After the `nff` package has been installed, we start by importing all dependencies for this tutorial.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import shutil
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.insert(0, '/home/jurgis/NeuralForceField/') # path towards the NFF folder for loading the module

import torch
from torch.optim import Adam
from torch.utils.data import DataLoader

from nff.data import Dataset, split_train_validation_test, collate_dicts, to_tensor
from nff.train import Trainer, get_trainer, get_model, load_model, loss, hooks, metrics, evaluate

from sigopt import Connection
from sigopt.examples import franke_function

## Loading the relevant data

As we usually work with the database, we can pack their information in a class `Dataset`, which is a subclass of `torch.utils.data.Dataset`. It basically wraps information on the atomic numbers, energies, forces and SMILES strings for each one of the geometries. In this example, we already have a pre-compiled `Dataset` to be used. We start by loading this file and creating three slices of the original dataset

In [3]:
dataset = Dataset.from_file('./data/dataset.pth.tar')

In [4]:
train, val, test = split_train_validation_test(dataset, val_size=0.2, test_size=0.2)

The `nff` code interfaces with the `graphbuilder` module through a git submodule in the repository. `graphbuilder` provides methods to create batches of graphs. In `nff`, we interface that through a custom dataloader called `
GraphLoader`. Here, we create one loader for each one of the slices.

In [5]:
train_loader = DataLoader(train, batch_size=50, collate_fn=collate_dicts)
val_loader = DataLoader(val, batch_size=50, collate_fn=collate_dicts)
test_loader = DataLoader(test, batch_size=50, collate_fn=collate_dicts)

In [7]:
from datetime import date
today = date.today()
variable = str(today.day) + str(today.month) + str(today.year)
path = '/home/jurgis/NeuralForceField/tutorials/sigopt/{}/'.format(variable)
if os.path.isdir(path) == False:
    os.mkdir(path)
else:
    print('Path aleardy exists')

Path aleardy exists


## Creating a model

`SIGOPT` is a standardized, scalable, optimization platform and API designed to unlock the potential of your modeling pipelines. This fully agnostic software solution accelerates, amplifies, and scales the model development process.

For registering on the website you need an invitation from someone in the lab.

Short introduction into the tools is given at https://app.sigopt.com/getstarted

In [8]:
conn = Connection(client_token="JQJLZYNHOWKBUXWMYBZFKRKHURZAZRIQWERJSBKWZUBODXEQ")

experiment = conn.experiments().create(
    name='tutorial ',
    metrics=[dict(name='energy', objective='minimize')],
    parameters=[
        dict(name='n_atom_basis', type='int', bounds=dict(min=120, max=360)),
        dict(name='n_filters', type='int', bounds=dict(min=120, max=360)),
        dict(name='n_gaussians', type='int', bounds=dict(min=10, max=70)),
        dict(name='n_convolutions', type='int', bounds=dict(min=1, max=10)),
        dict(name='cutoff', type='double', bounds=dict(min=1.0, max=10.0)),
    ],
    observation_budget = 10, # how many iterations to run for the optimization
)

DEVICE = 0

#### Model creation

Creating the model with `SIGOPT` is very similar to that of `nff`, one just wraps up the model part in a function and determines the optimization variable.

In [9]:
def create_model(assignments, suggestion_id, i = 0):
    params = dict()
    params['n_atom_basis'] = assignments['n_atom_basis']
    params['n_filters'] = assignments['n_filters'] 
    params['n_gaussians'] = assignments['n_gaussians'] 
    params['n_convolutions'] = assignments['n_convolutions'] 
    params['cutoff'] = assignments['cutoff']
    params['trainable_gauss'] = True 

    print(params)
    
    model = get_model(params)

    loss_fn = loss.build_mse_loss(loss_coef={'energy': 0.01, 'energy_grad': 1})

    trainable_params = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = Adam(trainable_params, lr=3e-4)

    train_metrics = [
        metrics.MeanAbsoluteError('energy'),
        metrics.MeanAbsoluteError('energy_grad')
    ]
    
    OUTDIR = path + str(suggestion_id)
    
    train_hooks = [
        hooks.MaxEpochHook(100),
        hooks.CSVHook(
            OUTDIR,
            metrics=train_metrics,
        ),
        hooks.PrintingHook(
            OUTDIR,
            metrics=train_metrics,
            separator = ' | ',
            time_strf='%M:%S'
        ),
        hooks.ReduceLROnPlateauHook(
            optimizer=optimizer,
            patience=30,
            factor=0.5,
            min_lr=1e-7,
            window_length=1,
            stop_after_min=True
        )
    ]
    
    T = Trainer(
        model_path=OUTDIR,
        model=model,
        loss_fn=loss_fn,
        optimizer=optimizer,
        train_loader=train_loader,
        validation_loader=val_loader,
        checkpoint_interval=1,
        hooks=train_hooks
    )

    T.train(device=DEVICE, n_epochs=10)
    
    _, _, val_loss = evaluate(T.get_best_model(), test_loader, loss_fn, device=DEVICE)
    
    return val_loss

def evaluate_model(assignments, i, suggestion_id):
    value = create_model(assignments=assignments, i=i, suggestion_id=suggestion_id)
    return value

In [10]:
i = 0

while experiment.progress.observation_count < experiment.observation_budget:

    suggestion = conn.experiments(experiment.id).suggestions().create()


    value = evaluate_model(assignments=suggestion.assignments, i=i, suggestion_id=suggestion.id)
    print (value)

    conn.experiments(experiment.id).observations().create(
      suggestion=suggestion.id,
      value=value,
    )


    experiment = conn.experiments(experiment.id).fetch()

{'n_atom_basis': 321, 'n_filters': 142, 'n_gaussians': 19, 'n_convolutions': 10, 'cutoff': 4.096756541252811, 'trainable_gauss': True}
 Time | Epoch | Learning rate | Train loss | Validation loss | MAE_energy | MAE_energy_grad | GPU Memory (MB)
20:19 |     1 |     3.000e-04 |     0.0000 |        283.4901 |    11.7943 |         11.6579 |             453
20:20 |     2 |     3.000e-04 |     0.0000 |        130.1209 |     8.9165 |          8.5261 |             453
20:21 |     3 |     3.000e-04 |     0.0000 |         79.9222 |     6.2299 |          6.4821 |             453
20:23 |     4 |     3.000e-04 |     0.0000 |         53.2885 |     3.1344 |          5.3414 |             453
20:24 |     5 |     3.000e-04 |     0.0000 |         39.3358 |     6.5732 |          4.4261 |             453
20:26 |     6 |     3.000e-04 |     0.0000 |         36.3659 |     3.2553 |          4.3964 |             453
20:27 |     7 |     3.000e-04 |     0.0000 |         35.6962 |     4.8125 |          4.3751 |  

21:35 |     1 |     3.000e-04 |     0.0000 |        326.3916 |    25.3239 |         12.5923 |             560
21:36 |     2 |     3.000e-04 |     0.0000 |        122.4115 |     1.7842 |          8.2101 |             560
21:38 |     3 |     3.000e-04 |     0.0000 |         69.9200 |     5.3672 |          5.9703 |             560
21:39 |     4 |     3.000e-04 |     0.0000 |         49.5380 |     1.3919 |          4.9472 |             560
21:40 |     5 |     3.000e-04 |     0.0000 |         37.6265 |     1.1964 |          4.3236 |             560
21:42 |     6 |     3.000e-04 |     0.0000 |         31.4068 |     0.9466 |          3.9467 |             560
21:43 |     7 |     3.000e-04 |     0.0000 |         27.5487 |     0.9138 |          3.7058 |             560
21:44 |     8 |     3.000e-04 |     0.0000 |         24.4636 |     0.8747 |          3.5191 |             560
21:46 |     9 |     3.000e-04 |     0.0000 |         21.5951 |     1.0873 |          3.3074 |             560
21:47 |   

### After the model finishes, all the results are viewable on SIGOPT website for data clarification